In [1]:
# Import all necessary libraries

import numpy as np
import pandas as pd
import scipy as sp
import scipy.io
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from six.moves import cPickle as pickle
import psutil
from sklearn.svm import SVC

%matplotlib inline

/Users/jeremy/anaconda/envs/venv/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [23]:
# Preprocessing functions

def get_indicies(matrix):
    z_len = np.shape(matrix)[2]
    indicies = []
    for i in range(z_len):
        slice = matrix[:,:,max(0, i - 1):i]
        slice = np.sum(slice)
        if slice != 0:
            indicies.append(i)
    return (min(indicies), max(indicies))

def get_crop_size(matrix):
    x = np.shape(matrix)[1]
    y = np.shape(matrix)[0]
    length_before_edge = 0
    edges = []
    
    for i in range(x):
        for j in range(y):
            if matrix[i][j] != 0:
                edges.append(length_before_edge)
                length_before_edge = 0
                break
            length_before_edge += 1
        length_before_edge = 0
    return max(edges), min(edges)

def save_1d_matrix(name, matrix, z_min, z_max, func=np.mean):
    data_dir = 'dat/'
    matrix = matrix[:,:,:z_min:z_max]
    matrix = func(matrix, axis=2)
    with open(data_dir + name, 'wb+') as f:
        pickle.dump(matrix, f, pickle.HIGHEST_PROTOCOL)
        
def merge_datasets(pickle_files, train_size, valid_size=0, remove_similar=False):
    num_classes = len(pickle_files)
    valid_dataset, valid_labels = make_arrays(valid_size, image_size)
    train_dataset, train_labels = make_arrays(train_size, image_size)
    vsize_per_class = valid_size // num_classes
    tsize_per_class = train_size // num_classes

    start_v, start_t = 0, 0
    end_v, end_t = vsize_per_class, tsize_per_class
    end_l = vsize_per_class + tsize_per_class
    for label, pickle_file in enumerate(pickle_files):
        try:
            with open(pickle_file, 'rb') as f:
                letter_set = pickle.load(f)
                # let's shuffle the letters to have random validation and training set
                np.random.shuffle(letter_set)
                if remove_similar and label != 8:
                    letter_set, _ = remove_similars(letter_set)
                if valid_dataset is not None:
                    valid_letter_set = letter_set[:vsize_per_class, :, :]
                    valid_dataset[start_v:end_v, :, :] = valid_letter_set
                    valid_labels[start_v:end_v] = label
                    start_v += vsize_per_class
                    end_v += vsize_per_class

                train_letter_set = letter_set[vsize_per_class:end_l, :, :]
                train_dataset[start_t:end_t, :, :] = train_letter_set
                train_labels[start_t:end_t] = label
                start_t += tsize_per_class
                end_t += tsize_per_class
        except Exception as e:
            print('Unable to process data from', pickle_file, ':', e)
            raise

    return valid_dataset, valid_labels, train_dataset, train_labels

In [ ]:
# Preprocessing all the data in batches
# Make sure all the files are .mat. Slice if necessary

mris = [f for f in os.listdir('../../mri/')]
segs = [f for f in os.listdir('../../seg/')][1:]

mri={}
seg={}
file_len = len(mris)
batch_amount = 50
end = 50

# TODO: Determine batch amount base off of available memory psutil.virtual_memory()

for i in range(0, file_len, batch_amount):
    # Loads seg knee mris
    if i > file_len:
        break
    if end > file_len:
        end = file_len
    for j in range(i, end):
        mri[mris[j]] = scipy.io.loadmat('../../mri/' + mris[j])['im_store']
        seg[segs[j]] = scipy.io.loadmat('../../seg/' + segs[j])['pred_con_vol']
    
    # Saves flattened knee mri as a pickle file
    for segmentation in segs:
        filename = segmentation.strip('.mat')
        z_min, z_max = get_indicies(seg[segmentation])
        save_1d_matrix(filename, mri[segmentation], z_min, z_max)
        
    end += batch_amount

# Frees up memory
mri = None; seg = None

In [7]:
# Loading data

pickles = [f for f in os.listdir('dat/')]
data = {}

for filename in pickles:
    with open('dat/' + filename, 'rb') as p:
        x = pickle.load(p).ravel()
        data[filename] = x
        
df = pd.DataFrame(data).T

# TODO: Match each mri to the right values
# df['hasEdema'] = []



In [ ]:
# SVM

train_size = 0
valid_size = 0

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
    pickles, train_size, valid_size, remove_similar=False)

#_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size, remove_similar=True)

edema_svm = SVC()

print ('Training a svm model..')
edema_svm.fit(x_train, y_train)

#Training Accuracy
training_accuracy=edema_svm.score(x_train,y_train)
print ('Training Accuracy:',training_accuracy)


validation_accuracy=edema_svm.score(x_test,y_test)
print('Accuracy of the model on unseen validation data: ',validation_accuracy)

In [ ]:
# DNN

